
# Glue Studio Notebook
You are now running a **Glue Studio** notebook; before you can start using your notebook you *must* start an interactive session.

## Available Magics
|          Magic              |   Type       |                                                                        Description                                                                        |
|-----------------------------|--------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------|
| %%configure                 |  Dictionary  |  A json-formatted dictionary consisting of all configuration parameters for a session. Each parameter can be specified here or through individual magics. |
| %profile                    |  String      |  Specify a profile in your aws configuration to use as the credentials provider.                                                                          |
| %iam_role                   |  String      |  Specify an IAM role to execute your session with.                                                                                                        |
| %region                     |  String      |  Specify the AWS region in which to initialize a session                                                                                                  |
| %session_id                 |  String      |  Returns the session ID for the running session.                                                                                                          |
| %connections                |  List        |  Specify a comma separated list of connections to use in the session.                                                                                     |
| %additional_python_modules  |  List        |  Comma separated list of pip packages, s3 paths or private pip arguments.                                                                                 |
| %extra_py_files             |  List        |  Comma separated list of additional Python files from S3.                                                                                                 |
| %extra_jars                 |  List        |  Comma separated list of additional Jars to include in the cluster.                                                                                       |
| %number_of_workers          |  Integer     |  The number of workers of a defined worker_type that are allocated when a job runs. worker_type must be set too.                                          |
| %worker_type                |  String      |  Standard, G.1X, *or* G.2X. number_of_workers must be set too. Default is G.1X                                                                            |
| %glue_version               |  String      |  The version of Glue to be used by this session. Currently, the only valid options are 2.0 and 3.0 (eg: %glue_version 2.0)                                |
| %security_config            |  String      |  Define a security configuration to be used with this session.                                                                                            |
| %sql                        |  String      |  Run SQL code. All lines after the initial %%sql magic will be passed as part of the SQL code.                                                            |
| %streaming                  |  String      |  Changes the session type to Glue Streaming.                                                                                                              |
| %etl                        |  String      |   Changes the session type to Glue ETL.                                                                                                                   |
| %status                     |              |  Returns the status of the current Glue session including its duration, configuration and executing user / role.                                          |
| %stop_session               |              |  Stops the current session.                                                                                                                               |
| %list_sessions              |              |  Lists all currently running sessions by name and ID.                                                                                                     |
| %spark_conf                 |  String      |  Specify custom spark configurations for your session. E.g. %spark_conf spark.serializer=org.apache.spark.serializer.KryoSerializer                       |

In [20]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

In [21]:
from setuptools import setup

setup(
    name="glue_python_shell_sample_module",
    version="0.1",
    install_requires=[
        "pyarrow~=0.15.1",
        "s3fs~=0.4.0"
    ]
) 

SystemExit: usage: 3342368526108747480 [global_opts] cmd1 [cmd1_opts] [cmd2 [cmd2_opts] ...]
   or: 3342368526108747480 --help [cmd1 cmd2 ...]
   or: 3342368526108747480 --help-commands
   or: 3342368526108747480 cmd --help

error: no commands supplied


In [22]:
import pandas as pd
from pathlib import Path
import re

In [23]:
bucket = "proyectointegrador"

## Datos homicidio

Cargamos y limpiamos todos los data sets relacionados con los homicidios

### Se leen fuentes de S3 y se agregan a una lista:

In [24]:
data_sources = []

for year in range(2010, 2022):

    df = pd.read_csv(f"s3://{bucket}/raw/h_{str(year)}/homicidios_{str(year)}.csv").dropna()
    data_sources.append(df)
    
data_sources[1]

       DEPARTAMENTO          MUNICIPIO  ...  AGRUPA EDAD PERSONA CANTIDAD
0         ANTIOQUIA              AMAGÁ  ...              ADULTOS        1
1         ANTIOQUIA      MEDELLÍN (CT)  ...         ADOLESCENTES        1
2         ANTIOQUIA      MEDELLÍN (CT)  ...              ADULTOS        1
3         ANTIOQUIA              TURBO  ...              ADULTOS        1
4         ATLÁNTICO  BARRANQUILLA (CT)  ...         ADOLESCENTES        1
...             ...                ...  ...                  ...      ...
11755         VALLE            PALMIRA  ...              ADULTOS        1
11756         VALLE            SEVILLA  ...              ADULTOS        1
11757        CALDAS            VITERBO  ...              ADULTOS        1
11758  CUNDINAMARCA   BOGOTÁ D.C. (CT)  ...              ADULTOS        1
11759        TOLIMA        IBAGUÉ (CT)  ...              ADULTOS        1

[11760 rows x 8 columns]


### Estandarizamos nombres de columnas

In [25]:
cols = []
for df in data_sources:
    for c in df.columns:
        cols.append(c)
    
count = {}
for c in cols:
    if c not in count:
        count[c] = 1
    else:
        count[c] += 1
        
count

{'DEPARTAMENTO': 11, 'MUNICIPIO': 12, 'CODIGO DANE': 11, 'ARMAS MEDIOS': 11, 'FECHA HECHO': 11, 'GENERO': 12, '*AGRUPA EDAD PERSONA': 8, 'CANTIDAD': 11, 'AGRUPA EDAD PERSONA': 3, 'ARMAS_MEDIOS': 1, 'DEPARTAMENTO ': 1, 'FECHA ': 1, '*AGRUPA_EDAD_PERSONA': 1, 'CODIGO_DANE': 1, 'CANTIDAD ': 1}


In [26]:
regex = re.compile('[^a-zA-Z]')
conversions = {'fecha':'fechahecho', 'agrupaedad':'agrupaedadpersona'}

for df in data_sources:
    # Sin caracteres especiales y en minusculas
    df.columns = [regex.sub('', c).lower() for c in df.columns]
    
    # Change column name to convention
    df.columns = [conversions[c] if c in conversions else c for c in df.columns]

    # Drop unnecesary columns
    df.drop(['agrupaedadpersona', 'armasmedios', 'genero'], axis=1, inplace=True)

    # Codigo DANE to int and remove 000 at the end
    df['codigodane'] = df['codigodane'].astype(str).apply(lambda x: int(int(x)/1000))


### Ordenamos columnas en orden alfabÃÂ©tico

In [27]:
for i in range(len(data_sources)):
    data_sources[i] = data_sources[i].reindex(sorted(data_sources[i].columns), axis=1)
    
data_sources[3]

       cantidad  codigodane departamento  fechahecho            municipio
0             1        5093    ANTIOQUIA  2013-01-01              BETULIA
1             1        5113    ANTIOQUIA  2013-01-01             BURITICÁ
2             1        5001    ANTIOQUIA  2013-01-01        MEDELLÍN (CT)
3             1        8758    ATLÁNTICO  2013-01-01              SOLEDAD
4             3       13001      BOLÍVAR  2013-01-01       CARTAGENA (CT)
...         ...         ...          ...         ...                  ...
11718         1       76111        VALLE  2013-12-31  GUADALAJARA DE BUGA
11719         1        5001    ANTIOQUIA  2013-12-31        MEDELLÍN (CT)
11720         1       18001      CAQUETÁ  2013-12-31       FLORENCIA (CT)
11721         1       18410      CAQUETÁ  2013-12-31         LA MONTAÑITA
11722         1       66400    RISARALDA  2013-12-31          LA VIRGINIA

[11723 rows x 5 columns]


In [28]:
length = 0
for df in data_sources:
    length += len(df)
length

134985


### Agregamos todos los datos a un mismo dataframe

In [29]:
# Crear un ÃÂºnico dataframe con todos los homicidios de todos los aÃÂ±os

data = data_sources[0]

for df in data_sources[1:]:
    data = data.append(df, ignore_index=True)
    
data

        cantidad  codigodane departamento  fechahecho          municipio
0              1        5030    ANTIOQUIA  2010-01-01              AMAGÁ
1              1        5001    ANTIOQUIA  2010-01-01      MEDELLÍN (CT)
2              1        5615    ANTIOQUIA  2010-01-01           RIONEGRO
3              1        8001    ATLÁNTICO  2010-01-01  BARRANQUILLA (CT)
4              1        8433    ATLÁNTICO  2010-01-01            MALAMBO
...          ...         ...          ...         ...                ...
134980         1       76520        VALLE  2021-12-08            PALMIRA
134981         1       76606        VALLE  2021-10-08           RESTREPO
134982         1       76834        VALLE  2021-05-08              TULUÁ
134983         1       76890        VALLE  2021-06-06             YOTOCO
134984         1       76892        VALLE  2021-12-11              YUMBO

[134985 rows x 5 columns]


In [30]:
df_cod_dep_mun = data[['codigodane','departamento','municipio']]
df_cod_dep_mun

        codigodane departamento          municipio
0             5030    ANTIOQUIA              AMAGÁ
1             5001    ANTIOQUIA      MEDELLÍN (CT)
2             5615    ANTIOQUIA           RIONEGRO
3             8001    ATLÁNTICO  BARRANQUILLA (CT)
4             8433    ATLÁNTICO            MALAMBO
...            ...          ...                ...
134980       76520        VALLE            PALMIRA
134981       76606        VALLE           RESTREPO
134982       76834        VALLE              TULUÁ
134983       76890        VALLE             YOTOCO
134984       76892        VALLE              YUMBO

[134985 rows x 3 columns]


### Agregamos columnas para dÃÂ­a, mes y aÃÂ±o

In [31]:
# Separamos la fecha en dÃÂ­a, mes y aÃÂ±o

data['fechahecho'] = data['fechahecho'].astype('datetime64[ns]')

data['day'] = data['fechahecho'].dt.day
data['month'] = data['fechahecho'].dt.month
data['year'] = data['fechahecho'].dt.year

data

        cantidad  codigodane departamento  ... day month  year
0              1        5030    ANTIOQUIA  ...   1     1  2010
1              1        5001    ANTIOQUIA  ...   1     1  2010
2              1        5615    ANTIOQUIA  ...   1     1  2010
3              1        8001    ATLÁNTICO  ...   1     1  2010
4              1        8433    ATLÁNTICO  ...   1     1  2010
...          ...         ...          ...  ...  ..   ...   ...
134980         1       76520        VALLE  ...   8    12  2021
134981         1       76606        VALLE  ...   8    10  2021
134982         1       76834        VALLE  ...   8     5  2021
134983         1       76890        VALLE  ...   6     6  2021
134984         1       76892        VALLE  ...  11    12  2021

[134985 rows x 8 columns]


### Agrupamos por municipio por aÃÂ±o; necesitamos los homicidios por cada aÃÂ±o de cada municipio

In [32]:
# Agrupamos por municipio y aÃÂ±o
data = data.groupby(['year','codigodane']).sum()
data

                 cantidad   day  month
year codigodane                       
2010 5001            1403  9581   4117
     5002               4    33     17
     5030              10   160     47
     5031               6    90     22
     5034              11   131     72
...                   ...   ...    ...
2021 95200              1    24      5
     97001              1    13      7
     99001              6    85     43
     99524              1    23     10
     99773              4    27     28

[9800 rows x 3 columns]


In [33]:
data.columns

Index(['cantidad', 'day', 'month'], dtype='object')


In [34]:
data = data.drop(["day","month"], axis=1)
data

                 cantidad
year codigodane          
2010 5001            1403
     5002               4
     5030              10
     5031               6
     5034              11
...                   ...
2021 95200              1
     97001              1
     99001              6
     99524              1
     99773              4

[9800 rows x 1 columns]


## Datos poblaciÃ³n

### Usamos los datos de poblaciÃÂ³n para crear un dataset con tasa de homicidio (a razÃÂ³n de la poblaciÃÂ³n) y no con valores totales

In [35]:
poblacion = pd.read_csv(f"s3://{bucket}/raw/Poblacion/Poblacion.csv", header=0, names=["year", "codigodane", "poblacion"], dtype={"year":"int64", "codigodane":"int64", "poblacion":"int64"}).dropna()
poblacion.dtypes

year          int64
codigodane    int64
poblacion     int64
dtype: object


In [36]:
poblacion

       year  codigodane  poblacion
0      1985        5001    1450177
1      1985        5002      27635
2      1985        5004       2851
3      1985        5021       4964
4      1985        5030      22541
...     ...         ...        ...
47119  2026       97889       1457
47120  2026       99001      21228
47121  2026       99524      10141
47122  2026       99624       4445
47123  2026       99773      84991

[47124 rows x 3 columns]


In [37]:
poblacion = poblacion.loc[poblacion['year'] > 2009]
poblacion = poblacion.loc[poblacion['year'] < 2022]
poblacion

       year  codigodane  poblacion
28050  2010        5001    2185539
28051  2010        5002      21822
28052  2010        5004       2655
28053  2010        5021       4679
28054  2010        5030      28132
...     ...         ...        ...
41509  2021       97889       1153
41510  2021       99001      20578
41511  2021       99524       9743
41512  2021       99624       4252
41513  2021       99773      79984

[13464 rows x 3 columns]


In [47]:
data_poblacion = data.merge(poblacion, on=["codigodane","year"])
data_poblacion

      codigodane  year  cantidad  poblacion
0           5001  2010      1403    2185539
1           5002  2010         4      21822
2           5030  2010        10      28132
3           5031  2010         6      23986
4           5034  2010        11      42152
...          ...   ...       ...        ...
9795       95200  2021         1       7194
9796       97001  2021         1      34307
9797       99001  2021         6      20578
9798       99524  2021         1       9743
9799       99773  2021         4      79984

[9800 rows x 4 columns]


In [48]:
data_poblacion["tasa"] = data_poblacion["cantidad"] * (100000 / data_poblacion["poblacion"].values)
data_poblacion

      codigodane  year  cantidad  poblacion       tasa
0           5001  2010      1403    2185539  64.194691
1           5002  2010         4      21822  18.330126
2           5030  2010        10      28132  35.546708
3           5031  2010         6      23986  25.014592
4           5034  2010        11      42152  26.096033
...          ...   ...       ...        ...        ...
9795       95200  2021         1       7194  13.900473
9796       97001  2021         1      34307   2.914857
9797       99001  2021         6      20578  29.157353
9798       99524  2021         1       9743  10.263779
9799       99773  2021         4      79984   5.001000

[9800 rows x 5 columns]


# Datos codigo municipio y cod dept

In [74]:
df_cod_dep_mun = pd.read_csv('s3://proyectointegrador/raw/MunDeptCod/MunDeptCod.csv', index_col=0)
df_cod_dep_mun.columns = ['departamento', 'dpto', 'codigodane', 'municipio']
df_cod_dep_mun = df_cod_dep_mun.drop_duplicates().reset_index().drop(['index'], axis=1)
df_cod_dep_mun

            departamento  dpto  codigodane      municipio
0              Antioquia     5        5001  Medellín (ct)
1              Antioquia     5        5002      Abejorral
2              Antioquia     5        5030          Amagá
3              Antioquia     5        5031         Amalfi
4              Antioquia     5        5034          Andes
...                  ...   ...         ...            ...
1067  Norte de santander    54       54347         Herrán
1068           Santander    68       68344           Hato
1069           Atlántico     8        8675    Santa lucía
1070              Boyacá    15       15550          Pisba
1071               Sucre    70       70670           Samp

[1072 rows x 4 columns]


In [75]:
data_poblacion_dpto = data_poblacion.merge(df_cod_dep_mun, on=['codigodane'], how='left',)
data_poblacion_dpto.reindex()


      codigodane  year  cantidad  ...  departamento  dpto            municipio
0           5001  2010      1403  ...     Antioquia   5.0        Medellín (ct)
1           5002  2010         4  ...     Antioquia   5.0            Abejorral
2           5030  2010        10  ...     Antioquia   5.0                Amagá
3           5031  2010         6  ...     Antioquia   5.0               Amalfi
4           5034  2010        11  ...     Antioquia   5.0                Andes
...          ...   ...       ...  ...           ...   ...                  ...
9807       95200  2021         1  ...      Guaviare  95.0           Miraflores
9808       97001  2021         1  ...        Vaupés  97.0            Mitú (ct)
9809       99001  2021         6  ...       Vichada  99.0  Puerto carreño (ct)
9810       99524  2021         1  ...       Vichada  99.0         La primavera
9811       99773  2021         4  ...       Vichada  99.0             Cumaribo

[9812 rows x 8 columns]


In [78]:
data_poblacion_dpto.columns

Index(['codigodane', 'year', 'cantidad', 'poblacion', 'tasa', 'departamento',
       'dpto', 'municipio'],
      dtype='object')


In [79]:
data_poblacion_dpto.columns = ['mpio_ccdgo','year','cantidad','poblacion','tasa','departamento', 'dpto', 'municipio_hecho']

In [80]:
data_poblacion.to_csv(f's3://{bucket}/trusted/data_poblacion.csv')